In [70]:
#Particle class from lecture with z-component added
class Particle:
    def __init__(self, x, y, z, vx, vy, vz, mass):
        self.x = x
        self.y = y
        self.z = z
        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.mass = mass
        
    def update(self, Fx, Fy, Fz, dt):
        self.vx = self.vx + Fx / self.mass * dt
        self.vy = self.vy + Fy / self.mass * dt
        self.vz = self.vz + Fz / self.mass * dt
        self.x = self.x + self.vx * dt
        self.y = self.y + self.vy * dt
        self.z = self.z + self.vz * dt
        
    def pairwise_force(self, particle):
        G = 6.674 * (10**-11)
        r2 = (self.x - particle.x)**2.0 + \
             (self.y - particle.y)**2.0 + \
             (self.z - particle.z)**2.0
        F_mag = -(G * particle.mass * self.mass * r2**.5)/(r2+100)**1.5
        F_x = (self.x - particle.x)/r2**0.5 * F_mag
        F_y = (self.y - particle.y)/r2**0.5 * F_mag
        F_z = (self.z - particle.z)/r2**0.5 * F_mag
        return (F_x, F_y, F_z)
    def distance(self, particle):
        return((self.x-particle.x)**2 + (self.y-particle.y)**2 + (self.z-particle.z)**2)**.5
    def print_particle(self):
        print("Position:", self.x, ",", self.y, ",", self.z)
        print("Velocity:", self.vx, ",", self.vy, ",", self.vz)

In [35]:
class Engine:
    def __init__(self, background, particles):
        self.background = background
        self.particles = particles
        self.t = 0.0
    def print_positions(self):
        for particle in self.particles:
            particle.print_particle()
            
    def plot_positions(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax = fig.gca(projection='3d')        
        ax.set_xlim(-10, 10)
        ax.set_ylim(-10, 10)
        ax.set_zlim(-10, 10)
        xs = []
        ys = []
        zs = []
        for particle in self.particles:
            xs.append(particle.x)
            ys.append(particle.y)
            zs.append(particle.z)
        ax.scatter(xs,ys,zs)
        name = 'fig'+str(self.t)+'.png'
        fig.savefig(name)
        plt.close(fig)
        
    def direct_euler(self, timestep):
        self.t += timestep
        for particle in self.particles:
            F_x_total = background[0]
            F_y_total = background[1]
            F_z_total = background[2]
            for other in self.particles:
                if other is particle:
                        continue
                F_x, F_y, F_z = particle.pairwise_force(other)
                F_x_total += F_x
                F_y_total += F_y
                F_z_total += F_z
            #print(F_x, F_y, F_z)
            particle.update(F_x_total, F_y_total, F_z_total, timestep)
            

In [57]:
import h5py
import numpy as np

############################
#NUMBER OF PARTICLES TO SIM#
############################
n = 3

# Create random data
particle_positions = np.random.uniform(-10, 10, size=(n, 3))
particle_velocities = np.random.uniform(0, 0, size=(n, 3))
particle_masses = np.random.uniform(1000000000000, 1000000000000, size=(n, 1))

# Write data to HDF5
try:
    data_file = h5py.File('dataset', 'w')
except:
    data_file = h5py.File('dataset', 'a')
    
data_file.create_dataset('particle_positions', data=particle_positions)
data_file.create_dataset('particle_velocities', data=particle_velocities)
data_file.create_dataset('particle_masses', data=particle_masses)
data_file.close()

In [75]:
import h5py
import matplotlib
import matplotlib.animation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib notebook

###################
#NAME OF HDF5 FILE#
###################
filename = 'dataset'
data_file = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % list(data_file.keys()))
a_group_key = list(data_file.keys())[0]

# Get the data
positions = list(data_file['particle_positions'])
velocities = list(data_file['particle_velocities'])
masses = list(data_file['particle_masses'])

particles = []
#Build particles from HDF5 data
for i in range (len(positions)):
    particle = Particle(positions[i][0], positions[i][1], positions[i][2], 
                        velocities[i][0], velocities[i][1], velocities[i][2],
                        masses[i])
    particles.append(particle)


    
background = [0,0,0]   
engine = Engine(background, particles)

engine.direct_euler(0)
engine.plot_positions()

runtime = 10.0
step = .5

while (runtime > engine.t):
    engine.direct_euler(step)
    engine.plot_positions()
    
data_file.close()

Keys: ['particle_masses', 'particle_positions', 'particle_velocities']
